In [3]:
# !pip install openpyxl

     ---------------------------------------- 0.0/250.0 kB ? eta -:--:--
     - -------------------------------------- 10.2/250.0 kB ? eta -:--:--
     ------ ------------------------------ 41.0/250.0 kB 326.8 kB/s eta 0:00:01
     ------------- ----------------------- 92.2/250.0 kB 581.0 kB/s eta 0:00:01
     ----------------- ------------------ 122.9/250.0 kB 654.9 kB/s eta 0:00:01
     ---------------------------- ------- 194.6/250.0 kB 737.3 kB/s eta 0:00:01
     ------------------------------------ 250.0/250.0 kB 807.2 kB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

The [Online Retail II](https://archive.ics.uci.edu/dataset/502/online+retail+ii) data set contains all the transactions occurring for a UK-based, registered, non-store online retail company between 01/12/2009 and 09/12/2011. They mainly sell unique all-occasion gift-ware. Many customers of the company are wholesalers.

**Additional variable information (from their website)**
- `InvoiceNo`: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation. 
- `StockCode`: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product. 
- `Description`: Product (item) name. Nominal. 
- `Quantity`: The quantities of each product (item) per transaction. Numeric.	
- `InvoiceDate`: Invoice date and time. Numeric. The day and time when a transaction was generated. 
- `UnitPrice`: Unit price. Numeric. Product price per unit in sterling (Â£). 
- `CustomerID`: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer. 
- `Country`: Country name. Nominal. The name of the country where a customer resides.

In [2]:
df_transactions_a = pd.read_excel('online_retail_II.xlsx', sheet_name="Year 2009-2010")
df_transactions_b = pd.read_excel('online_retail_II.xlsx', sheet_name="Year 2010-2011")
print(f"Dataframe shape: {df_transactions_a.shape}")
print(f"Dataframe shape: {df_transactions_b.shape}")

Dataframe shape: (525461, 8)
Dataframe shape: (541910, 8)


#### 0.0 **Preliminary EDA**

We notice that, for both datasets, there's a handful of missing values in the `Description` & `Customer ID` columns.

In [8]:
print(df_transactions_a.info())
print("")
df_transactions_a.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB
None



,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom


In [9]:
print(df_transactions_b.info())
print("")
df_transactions_b.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB
None



,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom


While comparisons of purchase habits between the 2 time periods is interesting, we will primarily focus on RFM & Customer Segmentation. Hence, we'll proceed to concatenate the 2 data sets together.

In [3]:
df_transactions = pd.concat([df_transactions_a, df_transactions_b], axis=0)
print(f"Dataframe shape: {df_transactions.shape}")

Dataframe shape: (1067371, 8)


Let's also ensure that each column has the correct data type. We'll treat `Description` & `Customer ID` after we deal with the missing values in the columns.

In [4]:
df_transactions[["Invoice", "StockCode", "Country"]] = \
  df_transactions[["Invoice", "StockCode", "Country"]].astype('string')

In [42]:
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1067371 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1067371 non-null  string        
 1   StockCode    1067371 non-null  string        
 2   Description  1062989 non-null  object        
 3   Quantity     1067371 non-null  int64         
 4   InvoiceDate  1067371 non-null  datetime64[ns]
 5   Price        1067371 non-null  float64       
 6   Customer ID  824364 non-null   float64       
 7   Country      1067371 non-null  string        
dtypes: datetime64[ns](1), float64(2), int64(1), object(1), string(3)
memory usage: 73.3+ MB


In [35]:
print(f"Transactions registered from {df_transactions['InvoiceDate'].min()} to {df_transactions['InvoiceDate'].max()}")

Transactions registered from 2009-12-01 07:45:00 to 2011-12-09 12:50:00


In [36]:
print(f"Number of transactions registered: {df_transactions['Invoice'].nunique()}")

Number of transactions registered: 53628


In [37]:
print(f"Number of item descriptions: {df_transactions['Description'].nunique()}")

Number of item descriptions: 5699


In [38]:
print(f"Number of item stock code: {df_transactions['StockCode'].nunique()}")

Number of item stock code: 5305


In [39]:
print(f"Number of unique customers: {df_transactions['Customer ID'].nunique()}")

Number of unique customers: 5942


When we look at the distributions of the `Quantity` & `Price` column, we see negative values which doesn't make sense. We'll address this in the next section.

In [51]:
# to avoid scientific notation
pd.set_option('display.float_format', '{:.2f}'.format)

df_transactions[["Quantity", "Price"]].describe()

,Quantity,Price
count,1067371.00,1067371.00
mean,9.94,4.65
std,172.71,123.55
min,-80995.00,-53594.36
25%,1.00,1.25
50%,3.00,2.10
75%,10.00,4.15
max,80995.00,38970.00


#### 1.0 **Data Cleaning**

##### 1.1 Addressing negative prices & quantities

0.6% of the data set has negative price points while 2% has negative recorded quantities.

More precisely:
- 99% of Invoices that are cancelled have negative `Quantity` rows. This perhaps represents returns, etc.
- 85% of transactions with negative quantities are cancelled invoices. We'll need to do further digging on what these other transactions represent.

In [72]:
(df_transactions["Price"] <= 0).mean() * 100

0.5815222635803297

In [73]:
(df_transactions["Quantity"] <= 0).mean() * 100

2.1501427338760375

In [76]:
df_transactions.loc[df_transactions["Quantity"] <= 0].tail(20)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
540082,C581464,71477,COLOURED GLASS STAR T-LIGHT HOLDER,-6,2011-12-08 18:57:00,3.95,15951.00,United Kingdom
540083,C581465,23660,HENRIETTA HEN MUG,-2,2011-12-08 18:59:00,1.65,15755.00,United Kingdom
540084,C581465,22171,3 HOOK PHOTO SHELF ANTIQUE WHITE,-1,2011-12-08 18:59:00,8.50,15755.00,United Kingdom
540085,C581465,21876,POTTERING MUG,-4,2011-12-08 18:59:00,1.65,15755.00,United Kingdom
540086,C581465,20914,SET/5 RED RETROSPOT LID GLASS BOWLS,-3,2011-12-08 18:59:00,2.95,15755.00,United Kingdom
540087,C581466,22838,3 TIER CAKE TIN RED AND CREAM,-1,2011-12-08 19:20:00,14.95,13883.00,United Kingdom
540088,C581466,22720,SET OF 3 CAKE TINS PANTRY DESIGN,-2,2011-12-08 19:20:00,4.95,13883.00,United Kingdom
540089,C581466,21216,"SET 3 RETROSPOT TEA,COFFEE,SUGAR",-1,2011-12-08 19:20:00,4.95,13883.00,United Kingdom
540090,C581466,21535,RED RETROSPOT SMALL MILK JUG,-2,2011-12-08 19:20:00,2.55,13883.00,United Kingdom
540091,C581466,21232,STRAWBERRY CERAMIC TRINKET POT,-1,2011-12-08 19:20:00,1.25,13883.00,United Kingdom


In [83]:
(df_transactions.loc[df_transactions["Quantity"] <= 0] 
                    ["Invoice"].str.startswith("C")).mean() * 100

84.93681917211329

In [81]:
(df_transactions.loc[df_transactions["Invoice"] \
                     .str.startswith("C")]["Quantity"] <= 0).mean() * 100

99.99487021647685

##### 1.2 Addressing the `Description` column null values

0.41 % percent of the rows in the fisrt sheet have missing values in the `Description` column. We also notice irregularities in the first 20 rows. All values in the `Price` column are 0 and some of the order quantities (`Quantity`) are negative. All rows in the `Customer ID` are also null.

We can also now confirm that all rows with their `Description` column empty have no `Customer ID`, are not cancellations. 61% of the rows also have negative quantities. Hence, they're potentially **return items** or **system logging errors**. This can explain why only some of their quantities are negative.

In [47]:
# calculate the percentage of missing values in the `Description` column
df_transactions["Description"].isnull().mean() * 100

0.4105414143723223

In [59]:
df_description_missing = df_transactions.loc[df_transactions["Description"].isnull()]
print(f"No. of missing rows: {df_description_missing.shape}")
print("")
df_description_missing.head(20)

No. of missing rows: (4382, 8)



,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.00,NaN,United Kingdom
3114,489655,20683,NaN,-44,2009-12-01 17:26:00,0.00,NaN,United Kingdom
3161,489659,21350,NaN,230,2009-12-01 17:39:00,0.00,NaN,United Kingdom
3731,489781,84292,NaN,17,2009-12-02 11:45:00,0.00,NaN,United Kingdom
4296,489806,18010,NaN,-770,2009-12-02 12:42:00,0.00,NaN,United Kingdom
4566,489821,85049G,NaN,-240,2009-12-02 13:25:00,0.00,NaN,United Kingdom
6378,489882,35751C,NaN,12,2009-12-02 16:22:00,0.00,NaN,United Kingdom
6555,489898,79323G,NaN,954,2009-12-03 09:40:00,0.00,NaN,United Kingdom
6576,489901,21098,NaN,-200,2009-12-03 09:47:00,0.00,NaN,United Kingdom
6581,489903,21166,NaN,48,2009-12-03 09:57:00,0.00,NaN,United Kingdom


In [64]:
# 61% of the rows have negative quantities.
(df_description_missing["Quantity"] <= 0).mean() * 100

61.364673664993155

In [57]:
# check if any of the rows are cancelled invoices
df_description_missing["Invoice"].str.startswith("c").sum()

0

In [53]:
# all the Customer IDs are missing
df_description_missing["Customer ID"].value_counts(normalize=True)

Series([], Name: proportion, dtype: float64)

In [52]:
# All price values are 0
df_description_missing["Price"].value_counts(normalize=True)

Price
0.00   1.00
Name: proportion, dtype: float64

In [60]:
df_description_missing["Country"].value_counts(normalize=True)

Country
United Kingdom   1.00
Name: proportion, dtype: Float64

##### 1.3 Addressing the `Customer ID` column null values

Overall, there is not a pattern or a clear reason fo why these rows don't have a `Customer ID` from the data. 1.7% of rows in the `Quantity` column are negative while this is 2.5% of rows in the `Price` column. Finally, none of these rows are cancelled transactions.

In [65]:
# calculate the percentage of missing values in the `Customer ID` column
df_transactions["Customer ID"].isnull().mean() * 100

22.766872999172733

In [66]:
df_customerid_missing = df_transactions.loc[df_transactions["Customer ID"].isnull()]
print(f"No. of missing rows: {df_customerid_missing.shape}")
print("")
df_customerid_missing.head(20)

No. of missing rows: (243007, 8)



,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
263,489464,21733,85123a mixed,-96,2009-12-01 10:52:00,0.00,NaN,United Kingdom
283,489463,71477,short,-240,2009-12-01 10:52:00,0.00,NaN,United Kingdom
284,489467,85123A,21733 mixed,-192,2009-12-01 10:53:00,0.00,NaN,United Kingdom
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.00,NaN,United Kingdom
577,489525,85226C,BLUE PULL BACK RACING CAR,1,2009-12-01 11:49:00,0.55,NaN,United Kingdom
578,489525,85227,SET/6 3D KIT CARDS FOR KIDS,1,2009-12-01 11:49:00,0.85,NaN,United Kingdom
1055,489548,22271,FELTCRAFT DOLL ROSIE,1,2009-12-01 12:32:00,2.95,NaN,United Kingdom
1056,489548,22254,FELT TOADSTOOL LARGE,12,2009-12-01 12:32:00,1.25,NaN,United Kingdom
1057,489548,22273,FELTCRAFT DOLL MOLLY,3,2009-12-01 12:32:00,2.95,NaN,United Kingdom
1058,489548,22195,LARGE HEART MEASURING SPOONS,1,2009-12-01 12:32:00,1.65,NaN,United Kingdom


In [71]:
# None of the rows are cancelled invoices
df_customerid_missing["Invoice"].str.startswith("c").sum()

0

In [68]:
df_customerid_missing["Description"].nunique()

4872

In [69]:
(df_customerid_missing["Quantity"] <= 0).mean() * 100

1.7308143386816017

In [70]:
(df_customerid_missing["Price"] <= 0).mean() * 100

2.5250301431646003

#### 2.0 **Conclusion**

Hence, we'll proceed with the following data treatments: 
- Remove rows where either `Quantity` or `Price` is null.
- Drop rows where the `Description` & `Customer ID` is missing.
- Remove `StockCode`s with `TEST` in their text as a cautionary step.
- Ensure data is between 01/12/2009 and 09/12/2011.

Future explorations can perhaps benefit to inspect if there's any meaningful differences between customers with an ID and those that don't.